In [ ]:
from PIL import Image, ImageDraw
import json


image_folder_path = '/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/DIP/final_project/defect-detection-and-segment-deep-learning/test/image/'
image_name_list = [image_folder_path + img_path for img_path in os.listdir(image_folder_path) if img_path.endswith('.png')]
image_name_list.sort()

label_folder_path = '/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/DIP/final_project/defect-detection-and-segment-deep-learning/test/label/'
label_name_list = [label_folder_path + label_path for label_path in os.listdir(label_folder_path) if label_path.endswith('.json')]
label_name_list.sort()

bbox_folder_path = '/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/DIP/final_project/defect-detection-and-segment-deep-learning/test/bbox/'

def draw_and_save_bbox(image_path:str, bbox_label:str, save_image:bool=False):

    # Open the image
    image = Image.open(image_path)

    # Load the bounding box data from the JSON file
    with open(bbox_label) as f:
        data = json.load(f)

    if data['shapes'] != 1:
        for i in range(len(data['shapes'])):
            points = data['shapes'][i]['points']

            x1, y1 = points[0]
            x2, y2 = points[1]
            
            # convert to yolo format
            
            # Create an ImageDraw object
            draw = ImageDraw.Draw(image)

            
            # paint the bounding box
            draw.rectangle([x1, y1, x2, y2], outline='green', width=7)
            
            # filename
            filename = image_path.split('/')[-1]
            
            if save_image == True:
            # save image
                image.save(bbox_folder_path + filename.replace('.png', '_bbox.png'))

for i in range(len(image_name_list)):
    print(image_name_list[i], label_name_list[i])
    draw_and_save_bbox(image_name_list[i], label_name_list[i], save_image=True)

In [ ]:
image_name_list.sort()
image_name_list

In [ ]:
label_name_list.sort()
label_name_list

### project target
* You are asked to detect and segment the defects of manufacturing from given images. 
* You are given with  total 450  images and ground truth annotations(mask and bounding box positions).
* You can use the image processing skills as well as deep learning methods for this homework.

### Project steps
* Step 1: Data preprocessing
* Step 2: Data augmentation
* Step 3: Model training: dectection and segmentation models
* Step 4: Model evaluation with IoU and precision
* Step 5: Creat a GUI for defect detection and segmentation

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sys
import os
import json
import numpy as np
from PIL import Image
from tqdm import  tqdm
import cv2



In [ ]:
# cover

In [ ]:

sys.path.append(os.path.realpath('..'))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

train_set_dir = '../defect-detection-and-segment-deep-learning/class_data/Train'
test_set_dir = '../defect-detection-and-segment-deep-learning/class_data/Val'

# Define the transformations to apply to the images
transform = torchvision.transforms.Compose([
    # resize the image to 224x224
    torchvision.transforms.Resize((224, 224)),
    
    # convert the rgb image to grayscale
    torchvision.transforms.Grayscale(num_output_channels=1),
    # convert the image to a tensor
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5,), (0.5,))
])

# Define the custom dataset class
class DefectDetectionDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None, mask:bool=False):
        self.root_dir = root_dir
        self.transform = transform
        self.mask = mask
        self.class_names = ['powder_uncover', 'powder_uneven', 'scratch']
        self.types = ['image']
        self.image_filenames = []
        self.mask_filenames = []
        self.bbox_filenames = []
        for class_name in tqdm(self.class_names):
            class_dir = os.path.join(root_dir, class_name)
            # concatenate the image, label and mask directories
            for type_name in self.types:
                type_dir = os.path.join(class_dir, type_name)
                for filename in os.listdir(type_dir):
                    if filename.endswith('.png'):
                        self.image_filenames.append(os.path.join(type_dir, filename))
                        self.mask_filenames.append(os.path.join(type_dir.replace('image', 'mask'), filename.replace('.png', '.png')))
                        self.bbox_filenames.append(os.path.join(type_dir.replace('image', 'label'), filename.replace('.png', '.json')))
    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image = Image.open(self.image_filenames[idx])
        mask = Image.open(self.mask_filenames[idx]).convert('L')

        with open(self.bbox_filenames[idx]) as f:
            bbox_data = json.load(f)

        for data in bbox_data['shapes']:
            

            # Extract the bounding box coordinates from the data
            x1, y1 = bbox_data['shapes']['points'][0]
            x2, y2 = bbox_data['shapes']['points'][1]

            # Convert the bounding box coordinates to relative values between 0 and 1
            img_width, img_height = image.size
            x1 /= img_width
            y1 /= img_height
            x2 /= img_width
            y2 /= img_height

        # Create the bounding box tuple
        bbox = (x1, y1, x2, y2)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        if self.mask:
            return image, mask, bbox
        else:
            return image, bbox
    
# Load the training and test datasets
trainset = DefectDetectionDataset(root_dir=train_set_dir, transform=transform)
testset = DefectDetectionDataset(root_dir=test_set_dir, transform=transform)

batch_size = 32

# Define the data loaders
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=1)

model = torchvision.models.resnet50()
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(trainset.class_names))

model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)




In [ ]:
trainset

In [ ]:
class_names = ['powder_uncover', 'powder_uneven', 'scratch']
types = ['image']
image_filenames = []
mask_filenames = []
bbox_filenames = []
root_dir = '../defect-detection-and-segment-deep-learning/class_data/Train'


for class_name in tqdm(class_names):
    class_dir = os.path.join(root_dir, class_name)
    # concatenate the image, label and mask directories
    for type_name in types:
        type_dir = os.path.join(class_dir, type_name)
        for filename in os.listdir(type_dir):
            if filename.endswith('.png'):
                image_filenames.append(os.path.join(type_dir, filename))
                mask_filenames.append(os.path.join(type_dir.replace('image', 'mask'), filename.replace('.png', '.png')))
                bbox_filenames.append(os.path.join(type_dir.replace('image', 'label'), filename.replace('.png', '.json')))




In [ ]:
bbox_filenames[0]

with open(bbox_filenames[0], "r") as file:
    data = json.load(file)
    print(data[-1])

In [34]:
import os, shutil
import cv2
from tqdm import tqdm
from PIL import Image
import numpy as np

# copy 

img_path = '/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/DIP/final_project/defect-detection-and-segment-deep-learning/pytorch-unet/output/scratch_converted_3152.png'
mask_path = '/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/DIP/final_project/defect-detection-and-segment-deep-learning/class_data/Val/scratch/mask/scratch_converted_3152.png'

img = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
# resize with .resize((1254, 1244))
mask = cv2.imread(mask_path,cv2.IMREAD_GRAYSCALE)

mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

print(img.shape)
print(mask.shape)

def dice_coef(y_true:np.ndarray, y_pred:np.ndarray):
    smooth = 1
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)

    return round((2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth),3)

print(dice_coef(y_true=mask, y_pred=img))

(1244, 1254)
(1244, 1254)
1.039


In [ ]:
import matplotlib.pyplot as plt
test = Image.open('/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/DIP/final_project/Pytorch-UNet/data/masks/powder_uncover_converted_0126.png').resize((1254, 1244))


test.save('/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/DIP/final_project/Pytorch-UNet/data/masks/powder_uncover_converted_0126.png')

In [ ]:
# print img size in img_list
for img in tqdm(img_list):
    print(img)
    img = cv2.imread(os.path.join(img_path, img))
    if img.shape != (1244, 1254, 3):
        print(img.shape)
        print('error')

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
from PIL import Image
%matplotlib inline



# resize mask

for img in tqdm(mask_list):
    img_file = Image.open(os.path.join(mask_path, img)).resize((1254, 1244))
    img_file.save(os.path.join(mask_path, img))
    

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
from PIL import Image
%matplotlib inline


for mask in tqdm(mask_list):
    mask_img = Image.open(os.path.join(mask_path, mask))
    mask_img = mask_img.convert('L')
    
    threshold = 10


    table = []
    for i in range(256):
        if i < threshold:
            table.append(0)
        else:
            table.append(1)
    photo = mask_img.point(table, '1')
    photo.save(os.path.join(mask_path, mask))

In [ ]:
# 图片二值化
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

# img = Image.open('/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/DIP/final_project/Pytorch-UNet/data/imgs/scratch_converted_3186.png')
img = Image.open('/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/DIP/final_project/Pytorch-UNet/data/imgs/powder_uneven_converted_0228.png')
print(Img.mode)
# 模式L”为灰色图像，它的每个像素用8个bit表示，0表示黑，255表示白，其他数字表示不同的灰度。
Img = img.convert('L')

plt.imshow(Img)
# 自定义灰度界限，大于这个值为黑色，小于这个值为白色
# threshold = 10

  
# table = []
# for i in range(256):
#   if i < threshold:
#     table.append(0)
#   else:
#     table.append(1)
  
# # 图片二值化
# photo = Img.point(table, '1')
# # photo.save('/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/DIP/final_project/Pytorch-UNet/data/masks/scratch_converted_3186.png')
# plt.imshow(photo)

In [ ]:

from IPython.display import display

img = Image.open('/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/DIP/final_project/Pytorch-UNet/power_uncover_converted_0126.png')
img = img.convert('L')
# save the image
img.save('/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/DIP/final_project/Pytorch-UNet/power_uncover_converted_0126.png')

In [ ]:
Image.open('/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/DIP/final_project/Pytorch-UNet/imgs copy/power_uneven_converted_0124.png').size

In [ ]:
import numpy as np
import cv2


input_img = cv2.imread('powder_uneven_converted_0199.png',  cv2.IMREAD_GRAYSCALE)
output_img =  cv2.imread('powder_uneven_converted_0199_out.png',  cv2.IMREAD_GRAYSCALE)

print(type(input_img))

def dice_coef(y_true:np.ndarray, y_pred:np.ndarray):
    smooth = 1
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)

    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)


In [ ]:
dice_coef(y_true=input_img, y_pred=output_img)

In [ ]:

def thresholding(img:np.ndarray, threshold:int):
    
    # convert to grayscale
    mask_img = img.convert('L')

    img_table = []
    
    # thresholding
    for i in range(256):
        if i < threshold:
            img_table.append(0)
        else:
            img_table.append(1)

    return mask_img.point(table, '1')

